In [38]:
from pyspark.sql import SparkSession
from pdb import set_trace
from pyspark.sql.functions import udf, col, size, expr
from pyspark.sql.types import ArrayType, StringType, FloatType, IntegerType, StructType, StructField, TimestampType
import json
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
import logging
import sys

In [39]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

PyTorch version: 2.4.1+cu121
CUDA available: True
GPU Device Name: NVIDIA GeForce GTX 1660


In [40]:
model_name = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"


In [41]:
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import logging
import sys

# Constants (Ensure these are correctly defined)
MODEL_FOLDER_PATH = "~/.cache/huggingface/hub/"  # Update this path
MODEL_NAME = "models--MoritzLaurer--deberta-v3-large-zeroshot-v2"
ENABLE_QUANTIZATION = False
BATCH_SIZE = 1  # Adjust based on your GPU capacity

# Step 2: Initialize Logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger("NLI_Inference")

# Initialize a global variable for the pipeline
nli_pipeline = None

def initialize_nli_pipeline(enable_quantization=False):
    global nli_pipeline
    try:
        # Load the tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # Load the model
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        
        if enable_quantization:
            model = torch.quantization.quantize_dynamic(
                model, {torch.nn.Linear}, dtype=torch.qint8
            )
            logger.info("Model quantization enabled.")
        else:
            logger.info("Model quantization disabled.")
        
        device = -1 if torch.cuda.is_available() else -1
        nli_pipeline = pipeline(
            task="zero-shot-classification",  # Changed task
            model=model,
            tokenizer=tokenizer,
            device=device
        )
        logger.info(f"NLI pipeline initialized on device: {'GPU' if device == 0 else 'CPU'}")
    except Exception as e:
        logger.error(f"Failed to initialize NLI pipeline: {e}")
        raise e

# Define the schema for the inference results
inference_schema = ArrayType(StructType([
    StructField("label", StringType(), False),
    StructField("score", FloatType(), False)
]))

@pandas_udf(inference_schema, PandasUDFType.SCALAR_ITER)
def md_inference_udf(iterator):
    global nli_pipeline
    if nli_pipeline is None:
        initialize_nli_pipeline(enable_quantization=ENABLE_QUANTIZATION)
    
    for batch_num, batch in enumerate(iterator, start=1):
        logger.info(f"Processing MD inference batch {batch_num} with {len(batch)} rows.")
        try:
            # Each 'batch' is a Pandas Series containing lists of text pairs
            # Flatten the list of text pairs in the batch
            flat_text_pairs = [pair for sublist in batch for pair in sublist]
            logger.debug(f"Batch {batch_num}: Total text pairs to infer: {len(flat_text_pairs)}")
            
            if len(flat_text_pairs):
                # Perform inference in batch using zero-shot-classification pipeline
                results = nli_pipeline(
                    sequences=flat_text_pairs,
                    candidate_labels=[
                        "This text is about consumer strength",
                        "This text is about consumer weakness",
                        "This text is about reduced consumer's spending patterns"
                    ],
                    multi_class=True,
                    batch_size=BATCH_SIZE
                )
                logger.debug(f"Batch {batch_num}: Inference completed with {len(results)} results.")
            else:
                results = []
            
            # Split results back to original rows
            split_results = []
            idx = 0
            for pairs in batch:
                if len(pairs):
                    # Extract the relevant slice of results
                    row_results = results[idx:idx+len(pairs)]
                    # Transform each result into a list of dicts with 'label' and 'score'
                    formatted_results = [
                        {"label": res['labels'][i], "score": res['scores'][i]}
                        for res in row_results
                        for i in range(len(res['labels']))
                    ]
                    split_results.append(formatted_results)
                    idx += len(pairs)
                else:
                    split_results.append([])
            
            yield pd.Series(split_results)
        except Exception as e:
            logger.error(f"Error in MD inference batch {batch_num}: {e}")
            # Yield empty results for this batch to continue processing
            yield pd.Series([[] for _ in batch])

In [42]:
# Sample data
data = [
    {
        "id": "1",
        "TEXT_PAIRS_MD": [
            ["Consumers are showing increased confidence in the market, leading to higher spending.<s><s>This text is about consumer strength"],
            ["There is a noticeable decline in consumer purchasing behavior this quarter.<s><s>This text is about consumer weakness"]
        ]
    }
]


In [43]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("NLI_Inference_Test") \
    .master("local[*]") \
    .getOrCreate()
spark.sparkContext.setLogLevel("INFO")

# Define the schema
schema = StructType([
    StructField("id", StringType(), False),
    StructField("TEXT_PAIRS_MD", ArrayType(ArrayType(StringType())), False)  # Array of arrays of strings
])

# Create the Spark DataFrame
sample_df = spark.createDataFrame(data, schema=schema)

# Display the DataFrame
sample_df.show(truncate=False)


24/09/25 11:48:47 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/09/25 11:48:47 INFO DAGScheduler: Got job 30 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/09/25 11:48:47 INFO DAGScheduler: Final stage: ResultStage 30 (showString at NativeMethodAccessorImpl.java:0)
24/09/25 11:48:47 INFO DAGScheduler: Parents of final stage: List()
24/09/25 11:48:47 INFO DAGScheduler: Missing parents: List()
24/09/25 11:48:47 INFO DAGScheduler: Submitting ResultStage 30 (MapPartitionsRDD[70] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/09/25 11:48:47 INFO MemoryStore: Block broadcast_30 stored as values in memory (estimated size 14.7 KiB, free 366.1 MiB)
24/09/25 11:48:47 INFO MemoryStore: Block broadcast_30_piece0 stored as bytes in memory (estimated size 7.2 KiB, free 366.1 MiB)
24/09/25 11:48:47 INFO BlockManagerInfo: Added broadcast_30_piece0 in memory on 192.168.1.6:41915 (size: 7.2 KiB, free: 36

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |TEXT_PAIRS_MD                                                                                                                                                                                                                                               |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[[Consumers are showing increased confidence in the market, leading to higher spending.<s><s>This text is about consumer strength], [There is a noticeable decline in consumer purchasing behavior this quarter.<s><s>This

24/09/25 11:48:47 INFO PythonRunner: Times: total = 42, boot = -13, init = 55, finish = 0
24/09/25 11:48:47 INFO Executor: Finished task 4.0 in stage 32.0 (TID 120). 1843 bytes result sent to driver
24/09/25 11:48:47 INFO PythonRunner: Times: total = 41, boot = 3, init = 38, finish = 0
24/09/25 11:48:47 INFO TaskSetManager: Finished task 4.0 in stage 32.0 (TID 120) in 54 ms on 192.168.1.6 (executor driver) (1/7)
24/09/25 11:48:47 INFO Executor: Finished task 1.0 in stage 32.0 (TID 117). 1886 bytes result sent to driver
24/09/25 11:48:47 INFO TaskSetManager: Finished task 1.0 in stage 32.0 (TID 117) in 58 ms on 192.168.1.6 (executor driver) (2/7)
24/09/25 11:48:47 INFO PythonRunner: Times: total = 55, boot = -1, init = 56, finish = 0
24/09/25 11:48:47 INFO PythonRunner: Times: total = 54, boot = 5, init = 49, finish = 0
24/09/25 11:48:47 INFO Executor: Finished task 3.0 in stage 32.0 (TID 119). 1843 bytes result sent to driver
24/09/25 11:48:47 INFO Executor: Finished task 2.0 in stage 

In [44]:
from pyspark.sql.functions import col

# Apply the UDF
result_df = sample_df.withColumn('MD_RESULT', md_inference_udf(col('TEXT_PAIRS_MD')))

# Display the results
result_df.select("id", "MD_RESULT").show(truncate=False)

24/09/25 11:48:47 INFO CodeGenerator: Code generated in 6.810579 ms
24/09/25 11:48:47 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/09/25 11:48:47 INFO DAGScheduler: Got job 33 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/09/25 11:48:47 INFO DAGScheduler: Final stage: ResultStage 33 (showString at NativeMethodAccessorImpl.java:0)
24/09/25 11:48:47 INFO DAGScheduler: Parents of final stage: List()
24/09/25 11:48:47 INFO DAGScheduler: Missing parents: List()
24/09/25 11:48:47 INFO DAGScheduler: Submitting ResultStage 33 (MapPartitionsRDD[75] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/09/25 11:48:47 INFO MemoryStore: Block broadcast_33 stored as values in memory (estimated size 33.9 KiB, free 366.0 MiB)
24/09/25 11:48:47 INFO MemoryStore: Block broadcast_33_piece0 stored as bytes in memory (estimated size 16.3 KiB, free 366.0 MiB)
24/09/25 11:48:47 INFO BlockManagerInfo: Added broadc

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |MD_RESULT                                                                                                                                                                                                                                                                                                                                                      |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

24/09/25 11:49:13 INFO ArrowPythonRunner: Times: total = 11323, boot = 365, init = 1, finish = 10957
24/09/25 11:49:13 INFO Executor: Finished task 6.0 in stage 35.0 (TID 134). 2354 bytes result sent to driver
24/09/25 11:49:13 INFO TaskSetManager: Finished task 6.0 in stage 35.0 (TID 134) in 11344 ms on 192.168.1.6 (executor driver) (7/7)
24/09/25 11:49:13 INFO TaskSchedulerImpl: Removed TaskSet 35.0, whose tasks have all completed, from pool 
24/09/25 11:49:13 INFO DAGScheduler: ResultStage 35 (showString at NativeMethodAccessorImpl.java:0) finished in 11.351 s
24/09/25 11:49:13 INFO DAGScheduler: Job 35 is finished. Cancelling potential speculative or zombie tasks for this job
24/09/25 11:49:13 INFO TaskSchedulerImpl: Killing all running tasks in stage 35: Stage finished
24/09/25 11:49:13 INFO DAGScheduler: Job 35 finished: showString at NativeMethodAccessorImpl.java:0, took 11.353248 s


In [45]:
result_df.select('TEXT_PAIRS_MD').show(5,truncate=False)

24/09/25 11:49:13 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/09/25 11:49:13 INFO DAGScheduler: Got job 36 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/09/25 11:49:13 INFO DAGScheduler: Final stage: ResultStage 36 (showString at NativeMethodAccessorImpl.java:0)
24/09/25 11:49:13 INFO DAGScheduler: Parents of final stage: List()
24/09/25 11:49:13 INFO DAGScheduler: Missing parents: List()
24/09/25 11:49:13 INFO DAGScheduler: Submitting ResultStage 36 (MapPartitionsRDD[77] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/09/25 11:49:13 INFO MemoryStore: Block broadcast_36 stored as values in memory (estimated size 14.4 KiB, free 366.2 MiB)
24/09/25 11:49:13 INFO MemoryStore: Block broadcast_36_piece0 stored as bytes in memory (estimated size 7.1 KiB, free 366.2 MiB)
24/09/25 11:49:13 INFO BlockManagerInfo: Added broadcast_36_piece0 in memory on 192.168.1.6:41915 (size: 7.1 KiB, free: 36

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|TEXT_PAIRS_MD                                                                                                                                                                                                                                               |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[Consumers are showing increased confidence in the market, leading to higher spending.<s><s>This text is about consumer strength], [There is a noticeable decline in consumer purchasing behavior this quarter.<s><s>This text is about c

24/09/25 11:49:13 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/09/25 11:49:13 INFO DAGScheduler: Got job 38 (showString at NativeMethodAccessorImpl.java:0) with 7 output partitions
24/09/25 11:49:13 INFO DAGScheduler: Final stage: ResultStage 38 (showString at NativeMethodAccessorImpl.java:0)
24/09/25 11:49:13 INFO DAGScheduler: Parents of final stage: List()
24/09/25 11:49:13 INFO DAGScheduler: Missing parents: List()
24/09/25 11:49:13 INFO DAGScheduler: Submitting ResultStage 38 (MapPartitionsRDD[77] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/09/25 11:49:13 INFO MemoryStore: Block broadcast_38 stored as values in memory (estimated size 14.4 KiB, free 366.1 MiB)
24/09/25 11:49:13 INFO MemoryStore: Block broadcast_38_piece0 stored as bytes in memory (estimated size 7.1 KiB, free 366.1 MiB)
24/09/25 11:49:13 INFO BlockManagerInfo: Added broadcast_38_piece0 in memory on 192.168.1.6:41915 (size: 7.1 KiB, free: 36